In [1]:
from minicons import scorer

/scratch/miniconda3/envs/dev/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
lm = scorer.IncrementalLMScorer("meta-llama/Llama-3.1-8B-Instruct", device="cuda:0")
# lm = scorer.IncrementalLMScorer("meta-llama/Llama-2-7b-chat-hf", device="cuda:0")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [47]:
def chat_template(text, response, tokenizer):
    formatted = [
        {"role": "system", "content": "Please negate what I say, as if you are talking to me, and not as if I am instructing you."},
        {"role": "user", "content": text},
        # {"role": "assistant", "content": response},
    ]
    string = tokenizer.apply_chat_template(
        formatted, tokenize=False, add_generation_prompt=True
    )
    return string

In [58]:
pre_prefix = chat_template(
    # "The actor, who has an interest in french cuisine, met the president!",
    "The actor, who met the president, has an interest in french cuisine!",
    "No, he did not.",
    tokenizer=lm.tokenizer,
) 

In [59]:
"""
hey wait a minute, no he does not.

that's not true, no he did not.
"""

prefix = [
    pre_prefix + "Hey, hang on a minute! No he",
    pre_prefix + "Hey, hang on a minute! No he",
    pre_prefix + "That's not true! No he",
    pre_prefix + "That's not true! No he",
]

continuations = [
    "didn't",
    "doesn't",
    "didn't",
    "doesn't"
]

In [60]:
lm.conditional_score(prefix, continuations)

[-4.237505912780762,
 -3.1448891162872314,
 -5.859060764312744,
 -5.278954029083252]

In [15]:
(-5.652661323547363/-6.450439453125) * (7771905/5085185)

1.3393201064336346

In [16]:
(-5.084205150604248/-5.893527507781982) * (7771905/5085185)

1.3184646112435188

In [ ]:
lm.conditional_score(prefix = ["he"]*2, stimuli= ["did", "does"])

[-6.603829860687256, -6.876806735992432]

In [19]:
pre_baseline = chat_template("", "", lm.tokenizer)

baseline = [
    pre_baseline + "Hey, wait a minute! No he",
    pre_baseline + "Hey, wait a minute! No he",
    pre_baseline + "That's not true! No he",
    pre_baseline + "That's not true! No he",
]

lm.conditional_score(baseline, ["did", "does", "did", "does"])

[-11.154016494750977,
 -11.976722717285156,
 -10.15646743774414,
 -11.547147750854492]

In [ ]:
(-5.652661323547363/-6.450439453125) * (7771905/5085185)
(-5.084205150604248/-5.893527507781982)

In [20]:
-5.652661323547363/-11.154016494750977

0.5067825860045552

In [21]:
-6.450439453125/-11.976722717285156

0.5385813469502418

In [22]:
-5.084205150604248/-10.15646743774414

0.5005879437677303

In [23]:
-5.893527507781982/-11.547147750854492

0.5103881612102746